In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

C:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#Read the datset
path_df='car_dataset.pickle'
with open(path_df, 'rb') as data:
    df = pickle.load(data)
df.head()
print(df.dtypes)

Url_Name       object
Content        object
Category       object
id              int64
News_length     int64
dtype: object


In [3]:
df.loc[0]['Content']

'\nImagine getting into your car, typing—or, better yet speaking—a location into your vehicle’s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. Self-driving vehicles—the stuff of science fiction since the first roads were paved—are coming, and they’re going to radically change what it’s like to get from point A to point B.\n \nIn 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google\'s self-driving project became Waymo.\n \nSince then, Waymo has invited the public to join the first public trial of autonomous vehicles operated by the Waymo Driver and introduced its first fully autonomous vehicles operated by the Waymo Driver on public roads without anyone in the driver’s seat.\n \n“The building blocks of driverless cars are on the road now,” explained Russ Rade

In [4]:
# #1. Text cleaning and preparation
# # \r and \n

# df['Content_Parsed_1'] = df['Content'].astype(str).str.replace("\n", " ")
# print(df.loc[0]['Content_Parsed_1'])
 
# df['Content_Parsed_2'] = df['Content_Parsed_1'].astype(str).str.replace("\n", " ")
# print(df.loc[0]['Content_Parsed_2'])

# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
punctuation_signs = list("?:!.,;")
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


In [5]:
# \r and \n
 
#df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content'].astype(str).str.replace("\n", " ")
print(df['Content_Parsed_1'])

0     Imagine getting into your car, typing—or, bet...
1      A self-driving car, also known as an autonom...
2    The continuing evolution of automotive technol...
3    To revist this article, visit My Profile, then...
4    In its race to embrace driverless vehicles, Wa...
Name: Content_Parsed_1, dtype: object


In [6]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)
df['Content_Parsed_5'] = lemmatized_text_list

In [7]:
stop_words = list(stopwords.words('english'))
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [9]:
df.loc[0]['Content']

'\nImagine getting into your car, typing—or, better yet speaking—a location into your vehicle’s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. Self-driving vehicles—the stuff of science fiction since the first roads were paved—are coming, and they’re going to radically change what it’s like to get from point A to point B.\n \nIn 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google\'s self-driving project became Waymo.\n \nSince then, Waymo has invited the public to join the first public trial of autonomous vehicles operated by the Waymo Driver and introduced its first fully autonomous vehicles operated by the Waymo Driver on public roads without anyone in the driver’s seat.\n \n“The building blocks of driverless cars are on the road now,” explained Russ Rade

In [10]:
df.loc[1]['Content_Parsed_1']

'  A self-driving car, also known as an autonomous vehicle (AV or auto), driverless car, or robo-car (robotic car),[1][2][3] is a vehicle that is capable of sensing its environment and moving safely with little or no human input.[4][5]  Self-driving cars combine a variety of sensors to perceive their surroundings, such as radar, lidar, sonar, GPS, odometry and inertial measurement units.[1][4] Advanced control systems interpret sensory information to identify appropriate navigation paths, as well as obstacles and relevant signage.[4][6][7][8]  Possible implementations of the technology include personal self-driving vehicles, shared robotaxis, connected vehicle platoons and long-distance trucking.[4] Several projects to develop a fully self-driving commercial car are in various stages of development. Waymo became the first service provider to offer robotaxi rides to the general public in a part of Phoenix, Arizona in 2020, while Tesla has said it will offer subscription-based "full self

In [11]:
df.loc[1]['Content_Parsed_2']

"  a self-driving car, also known as an autonomous vehicle (av or auto), driverless car, or robo-car (robotic car),[1][2][3] is a vehicle that is capable of sensing its environment and moving safely with little or no human input.[4][5]  self-driving cars combine a variety of sensors to perceive their surroundings, such as radar, lidar, sonar, gps, odometry and inertial measurement units.[1][4] advanced control systems interpret sensory information to identify appropriate navigation paths, as well as obstacles and relevant signage.[4][6][7][8]  possible implementations of the technology include personal self-driving vehicles, shared robotaxis, connected vehicle platoons and long-distance trucking.[4] several projects to develop a fully self-driving commercial car are in various stages of development. waymo became the first service provider to offer robotaxi rides to the general public in a part of phoenix, arizona in 2020, while tesla has said it will offer subscription-based full self-

In [12]:
df.loc[1]['Content_Parsed_3']

"  a self-driving car also known as an autonomous vehicle (av or auto) driverless car or robo-car (robotic car)[1][2][3] is a vehicle that is capable of sensing its environment and moving safely with little or no human input[4][5]  self-driving cars combine a variety of sensors to perceive their surroundings such as radar lidar sonar gps odometry and inertial measurement units[1][4] advanced control systems interpret sensory information to identify appropriate navigation paths as well as obstacles and relevant signage[4][6][7][8]  possible implementations of the technology include personal self-driving vehicles shared robotaxis connected vehicle platoons and long-distance trucking[4] several projects to develop a fully self-driving commercial car are in various stages of development waymo became the first service provider to offer robotaxi rides to the general public in a part of phoenix arizona in 2020 while tesla has said it will offer subscription-based full self-driving to private 

In [13]:
df.loc[1]['Content_Parsed_4']

"  a self-driving car also known as an autonomous vehicle (av or auto) driverless car or robo-car (robotic car)[1][2][3] is a vehicle that is capable of sensing its environment and moving safely with little or no human input[4][5]  self-driving cars combine a variety of sensors to perceive their surroundings such as radar lidar sonar gps odometry and inertial measurement units[1][4] advanced control systems interpret sensory information to identify appropriate navigation paths as well as obstacles and relevant signage[4][6][7][8]  possible implementations of the technology include personal self-driving vehicles shared robotaxis connected vehicle platoons and long-distance trucking[4] several projects to develop a fully self-driving commercial car are in various stages of development waymo became the first service provider to offer robotaxi rides to the general public in a part of phoenix arizona in 2020 while tesla has said it will offer subscription-based full self-driving to private 

In [14]:
df.loc[1]['Content_Parsed_6']

"   self-driving car also know   autonomous vehicle (av  auto) driverless car  robo-car (robotic car)[1][2][3]   vehicle   capable  sense  environment  move safely  little   human input[4][5]  self-driving cars combine  variety  sensors  perceive  surround   radar lidar sonar gps odometry  inertial measurement units[1][4] advance control systems interpret sensory information  identify appropriate navigation paths  well  obstacles  relevant signage[4][6][7][8]  possible implementations   technology include personal self-driving vehicles share robotaxis connect vehicle platoons  long-distance trucking[4] several project  develop  fully self-driving commercial car   various stag  development waymo become  first service provider  offer robotaxi rid   general public   part  phoenix arizona  2020  tesla  say   offer subscription-based full self-driving  private vehicle owners  2021[9][10]  nuro   allow  start autonomous commercial delivery operations  california  2021[11]   china two publicl

In [15]:
df.head(1)

,Url_Name,Content,Category,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,https://www.investopedia.com/articles/investin...,"\nImagine getting into your car, typing—or, be...",finance,1,14562,"Imagine getting into your car, typing—or, bet...","imagine getting into your car, typing—or, bet...",imagine getting into your car typing—or bette...,imagine getting into your car typing—or bette...,imagine get into your car typing—or better ye...,imagine get car typing— better yet speaking...


In [16]:
list_columns = ["Url_Name", "Category", "Content", "Content_Parsed_6"]
new_df = df[list_columns]
new_df = new_df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [17]:
new_df.head()

,Url_Name,Category,Content,Content_Parsed
0,https://www.investopedia.com/articles/investin...,finance,"\nImagine getting into your car, typing—or, be...",imagine get car typing— better yet speaking...
1,https://en.wikipedia.org/wiki/Self-driving_car,technology,"\n A self-driving car, also known as an autono...",self-driving car also know autonomous veh...
2,https://www.nhtsa.gov/technology-innovation/au...,technology,The continuing evolution of automotive technol...,continue evolution automotive technology aim...
3,https://www.wired.com/story/gm-driverless-toyo...,business,"To revist this article, visit My Profile, then...",revist article visit profile view save sto...
4,https://www.latimes.com/politics/la-na-pol-sel...,politics,"In its race to embrace driverless vehicles, Wa...",race embrace driverless vehicles washington...


In [18]:
category_codes = {
    'business': 0,
    'finance': 1,
    'politics': 2,
    'technology': 3
}

In [19]:
# Category mapping
new_df['Category_Code'] = df['Category']
new_df = new_df.replace({'Category_Code':category_codes})

In [20]:
new_df.head()

,Url_Name,Category,Content,Content_Parsed,Category_Code
0,https://www.investopedia.com/articles/investin...,finance,"\nImagine getting into your car, typing—or, be...",imagine get car typing— better yet speaking...,1
1,https://en.wikipedia.org/wiki/Self-driving_car,technology,"\n A self-driving car, also known as an autono...",self-driving car also know autonomous veh...,3
2,https://www.nhtsa.gov/technology-innovation/au...,technology,The continuing evolution of automotive technol...,continue evolution automotive technology aim...,3
3,https://www.wired.com/story/gm-driverless-toyo...,business,"To revist this article, visit My Profile, then...",revist article visit profile view save sto...,0
4,https://www.latimes.com/politics/la-na-pol-sel...,politics,"In its race to embrace driverless vehicles, Wa...",race embrace driverless vehicles washington...,2


In [22]:
X_train, X_test, y_train, y_test = train_test_split(new_df['Content_Parsed'], 
                                                    new_df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [36]:
# Parameter election
ngram_range = (1,2)
min_df = 4
max_df = 1.
max_features = 300

In [37]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [38]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(4, 56)
(1, 56)


In [39]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. safety
. driver
. could
. take
. drive
  . Most correlated bigrams:
. driving vehicles
. self driving

# 'finance' category:
  . Most correlated unigrams:
. design
. lose
. year
. test
. 2020
  . Most correlated bigrams:
. driving vehicles
. self driving

# 'politics' category:
  . Most correlated unigrams:
. government
. human
. industry
. year
. years
  . Most correlated bigrams:
. self driving
. driving vehicles

# 'technology' category:
  . Most correlated unigrams:
. cars
. take
. test
. year
. safety
  . Most correlated bigrams:
. driving vehicles
. self driving



In [40]:
bigrams

['driving vehicles', 'self driving']

In [41]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)